In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 15993632
paper_name = 'corbacho_hernandez_2005' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [11]:
original_data = pd.read_csv('raw_data/Corbacho_tables.txt', sep='\t', encoding = "ISO-8859-1", skiprows=2)

In [12]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 268 x 7


In [13]:
original_data.head()

,ORF,Gene name,Short description,Ldb group,Invertase size(a),,Unnamed: 6
0,YBL082C,RHK1/ALG3,"DolP-Man dependent alpha-1,3-mannosyl transfer...",3,R1,,NaN
1,YBR015C,MNN2/LDB8,"Golgi alpha-1,2-mannosyltransferase",1,R2,,NaN
2,YDL232W,OST4,3.6 kDa subunit of ER DolPP-oligosaccharide/pr...,1,R1,,NaN
3,YDR245W,MNN10,Subunit of Golgi mannan polymerase II (Man Pol...,1,R2,,NaN
4,YEL036C,ANP1/MNN8,Subunit of Golgi Man Pol II,1,R1,,NaN


In [16]:
original_data['orf'] = original_data['ORF '].astype(str)

In [17]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [18]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [19]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

                                                          ORF   Gene name   \
index_input                                                                  
19                                            (a) See Fig. 3.          NaN   
20                                                         NaN         NaN   
21                                                    Table 2          NaN   
22           Genes related to vacuolar acidifcation whose d...         NaN   
23                                                        ORF   Gene name    
40                                            (a) See Fig. 3.          NaN   
41                                                         NaN         NaN   
42                                                    Table 3                
43           Genes related to vacuolar targeting whose dele...               
44                                                        ORF   Gene name    
66                                            (a) See Fig. 3.   

In [20]:
original_data = original_data.loc[t,:]

In [23]:
original_data['data'] = original_data['Ldb group ']

/opt/anaconda3/envs/yp-data/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [25]:
original_data.loc[original_data['data']=='1 (alpha)','data'] = '1'

/opt/anaconda3/envs/yp-data/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/opt/anaconda3/envs/yp-data/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [26]:
original_data['data'] = pd.to_numeric(original_data['data'], errors='coerce')

/opt/anaconda3/envs/yp-data/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [28]:
original_data.set_index('orf', inplace=True)

In [29]:
original_data = original_data[['data']].copy()

In [30]:
original_data = original_data.groupby(original_data.index).mean()

In [32]:
original_data['data'] = original_data['data']-4

In [33]:
original_data.shape

(212, 1)

# Prepare the final dataset

In [34]:
data = original_data.copy()

In [35]:
dataset_ids = [582]
datasets = datasets.reindex(index=dataset_ids)

In [36]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [37]:
data.head()

dataset_id,582
data_type,value
orf,
YAL016W,-2
YAL047C,-2
YAL056W,-1
YBL006C,-2
YBL007C,-2


## Subset to the genes currently in SGD

In [38]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 1


In [39]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,582
,data_type,value
gene_id,orf,
14,YAL016W,-2
45,YAL047C,-2
52,YAL056W,-1
94,YBL006C,-2
95,YBL007C,-2


# Normalize

In [40]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [41]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [42]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id        582          
data_type       value    valuez
gene_id orf                    
14      YAL016W    -2 -5.310135
45      YAL047C    -2 -5.310135
52      YAL056W    -1 -2.655067
94      YBL006C    -2 -5.310135
95      YBL007C    -2 -5.310135

# Print out

In [43]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [44]:
from IO.save_data_to_db3 import *

In [45]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 15993632...
Inserting the new data...


100%|██████████| 1/1 [00:00<00:00,  1.81it/s]

Updating the data_modified_on field...
